In [8]:
from numpy import *

In [9]:
class treeNote():
    def __init__(self, feat, val, right, left):
        fetureToSplitOn = feat
        valueOfSplit = val
        rightBranch = right
        leftBranch = left

In [10]:
def binSplitDataSet(dataSet, feature, value):
    mat0 = dataSet[nonzero(dataSet[:,feature] > value)[0],:][0]
    mat1 = dataSet[nonzero(dataSet[:,feature] <= value)[0],:][0]
    return mat0, mat1

testMat = mat(eye(4))
mat0, mat1 = binSplitDataSet(testMat, 1, 0.5)
print mat0
print mat1

[[0. 1. 0. 0.]]
[[1. 0. 0. 0.]]


In [11]:
def regLeaf(dataSet):
    return mean(dataSet[:-1]) # 平均值

In [12]:
def regErr(dataSet):
    return var(dataSet[:,-1] * shape(dataSet))[0] # 总方差 * m

In [13]:
def chooseBestSplit(dataSet, leafType=regLeaf, errType=regErr, ops=(1,4)):
    tolS = ops[0]; tolN = opt[1]
    if len(set(dataSet[:,-1].T.tolist()[0])) == 1: # 只剩下一种分类了，停止
        return None, leafType(dataSet)

    m, n = shape(dataSet)
    S = errType(dataSet)
    
    beatS = inf; bestIndex = 0; bestValue = 0;
    for featIndex in range(n-1): # 对每个特征
        for splitVal in set(dataSet[:, featIndex]): #对于每个特征值
            mat0, mat1 = binSplitDataSet(dataSet, featIndex, splitVal)
            if (shape(mat0)[0] < tolN) or (shape(mat1)[0] < tolN): continue # 如果少于最小样本数
            newS = errType(mat0) + errType(mat1)
            if newS < bestS:
                bestIndex = featIndex
                bestValue = splitVal
                beatS = newS
    if (S - beatS) < tolS: # 误差的减少 小于阈值 则退出
        return None, leafType(dataSet)
    mat0, mat1 = binSplitDataSet(dataSet, bestIndex, bestValue)
    if (shape(mat0)[0] < tolN) or (shape(mat1)[0] < tolN):
        return None, leafType(dataSet)
    return bestIndex, bestValue

In [14]:
def createTree(dataSet, leafType=regLeaf, errType=regErr, ops=(1,4)):
    feat, val = chooseBestSplit(dataSet, leafType, errType, ops)
    if feat == None: return val
    retTree = {}
    retTree['spInd'] = feat
    retTree['spVal'] = val
    lSet, rSet = binSplitDataSet(dataSet, feat, val)
    retTree['left'] = createTree(lSet, leafType, errType, ops)
    retTree['right'] = createTree(rSet, leafType, errType, ops)

    return retTree

In [15]:
# 剪枝
def isTree(obj):
    return (type(obj).__name__ == 'dict')

def getMean(tree):
    if isTree(tree['right']): 
        tree['right'] = getMean(tree['right'])
    if isTree(tree['left']):
        tree['left'] = getMean(tree['left'])
    return (tree['left'] + tree['right']) / 2.0

def prune(tree, testData):
    if shape(testData)[0] == 0:
        return getMean(tree)
    
    if (isTree(tree['right']) or isTree(tree['left'])): # 划分样本
        lSet, rSet = binSplitDataSet(testData, tree['spInd'], tree['spVal']) 
        
    if isTree(tree['left']): # 递归剪枝
        tree['left'] = prune(tree['left'], lSet)
    if isTree(tree['right']):
        tree['right'] = prune(tree['right'], rSet)
    
    if not isTree(tree['left']) and not isTree(tree['right']):
        lSet, rSet = binSplitDataSet(testData, tree['spInd'], tree['spVal'])
        
        # 原始误差方差
        errorNoMerge = sum(power(lSet[:,-1] - tree['left'], 2)) + sum(power(rSet[:,-1] - tree['right'], 2))
        
        # 融合两个叶子节点后，平均值形成的误差
        treeMean = (tree['left'] + tree['right']) / 2.0 
        errorMerge = sum(power(testData[:-1] - treeMean, 2)) 
    
        if errorMerge < errorNoMerge:
            print "merging"
            return treeMean
        else:
            return tree
    else:
        return tree